In [1]:
import json
from datetime import datetime
from pprint import pprint
import numpy as np
import pandas as pd

import os
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import make_pipeline
from tqdm.notebook import tqdm

In [2]:
import heart, hotel, airline

train_size = 700
test_size = 300

data_and_transformers = [
    ('heart', heart.load(train_size, test_size), heart.binarizer()),
    ('airline', airline.load(train_size, test_size), airline.binarizer()),
    ('hotel', hotel.load(train_size, test_size), hotel.binarizer()),
]

In [3]:
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lazy_fca_sklearn_wrapper import MyBinarizedBinaryClassifier, MyPatternBinaryClassifier

models_and_param_grids = {
    'naive_bayes': (BernoulliNB(binarize=False), {
        'bernoullinb__alpha': [1.0,]
    }),
    'xgboost': (XGBClassifier(random_state=0, objective='binary:logistic'), {
        'xgbclassifier__n_estimators': [100,],
    }),
    'random_forest': (RandomForestClassifier(random_state=0), {
        'randomforestclassifier__n_estimators': [10,],
    }),
    'logreg': (LogisticRegression(random_state=0), {
        'logisticregression__C': [0.1,], 
    }),
    'knn': (KNeighborsClassifier(), {
        'kneighborsclassifier__n_neighbors': [5,],
        'kneighborsclassifier__weights': ['distance'],
    }),
    'catboost': (CatBoostClassifier(verbose=0, random_state=0), {
        'catboostclassifier__n_estimators': [100,],
    }),
    'lazy_fca': (MyBinarizedBinaryClassifier(), {
        'mybinarizedbinaryclassifier__alpha': [0, 0.1, 0.5, 0.9, 1.0],
        'mybinarizedbinaryclassifier__method': ['standard', 'standard-support', 'ratio-support'],
    }),
    # 'lazy_fca_pat_structures': (MyPatternBinaryClassifier(), {
    #     'mybinarizedbinaryclassifier__alpha': [0, 0.1, 0.5, 0.9, 1.0],
    #     'mybinarizedbinaryclassifier__method': ['standard', 'standard-support', 'ratio-support'],
    # })
}

In [4]:
from sklearn.pipeline import FunctionTransformer


results = []
experiment_dir = f'./output/{datetime.now()}'
os.makedirs(experiment_dir)

for data_name, data, transformers in tqdm(data_and_transformers):
    for model_name, (model, param_grid) in tqdm(models_and_param_grids.items()):
        print(data_name, model_name)
        
        if model_name == 'lazy_fca':
            to_bool = FunctionTransformer(
                lambda df: np.asarray(df).astype(bool),
                feature_names_out='one-to-one'
            )
            pipeline = make_pipeline(*transformers, to_bool, model)
        else:
            pipeline = make_pipeline(*transformers, model)
            
        grid_search = GridSearchCV(
            estimator=pipeline,
            cv=StratifiedKFold(n_splits=5),
            param_grid=param_grid,
            n_jobs=3,
            verbose=1,
            scoring=['accuracy', 'f1', 'precision', 'recall'],
            refit='f1',
            error_score='raise'
        )

        X_train, X_test, y_train, y_test = data
        if model_name == 'lazy_fca':
            y_train = np.asarray(y_train).astype(bool) 
            y_test = np.asarray(y_test).astype(bool) 
            
        grid_search.fit(X=X_train, y=y_train)
        y_pred = grid_search.predict(X_test)

        res = {
            'dataset': data_name,
            'model': model_name,
            'f1': f1_score(y_test, y_pred),
            'accuracy': accuracy_score(y_test, y_pred),
            'params': grid_search.best_params_
        }
        display(pd.DataFrame([res]))
        with open(f'{experiment_dir}/{model_name}_{data_name}.json', 'wt') as f:
            json.dump(res, f)
        results.append(res)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

heart naive_bayes
Fitting 5 folds for each of 1 candidates, totalling 5 fits


,dataset,model,f1,accuracy,params
0,heart,naive_bayes,0.266667,0.816667,{'bernoullinb__alpha': 1.0}


heart xgboost
Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

,dataset,model,f1,accuracy,params
0,hotel,catboost,0.639175,0.766667,{'catboostclassifier__n_estimators': 100}


/home/alex/python_venv/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


hotel lazy_fca
Fitting 5 folds for each of 15 candidates, totalling 75 fits


/home/alex/python_venv/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/home/alex/python_venv/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/home/alex/python_venv/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/home/alex/python_venv/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/home/alex/python_venv/lib/python3.8/site-packages/sklearn/preproces

,dataset,model,f1,accuracy,params
0,hotel,lazy_fca,0.651376,0.746667,"{'mybinarizedbinaryclassifier__alpha': 1.0, 'm..."


In [5]:
df_results = pd.DataFrame(results).pivot(
    index='model',
    columns='dataset',
    values=['f1', 'accuracy'],
).swaplevel(axis=1).sort_index(axis=1)
df_results.style.highlight_max(color='red').format(precision=3)

In [6]:
df_results.style.highlight_max(props='textbf:--rwrap;').format(precision=3).to_latex('table.tex')